In [1]:
!pip install factor-analyzer


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for factor-analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42568 sha256=213a9f23f9fc171f25bf0f44e3f41cb3fd2f02265187cfb19445df12715b0da6
  Stored in directory: /Users/tokamohamed/Library/Caches/pip/wheels/fa/f7/53/a55a8a56668a6fe0199e0e02b6e0ae3007ec35cdf6e4c25df7
Successfully built factor-analyzer


In [ ]:
from factor_analyzer.factor_analyzer import FactorAnalyzer 
fa = FactorAnalyzer(rotation='varimax')
fa.fit(df)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
ev